In [6]:
import json
from pathlib import Path
from typing import List, Optional, Dict, Any, Union
from peewee import *
import os

In [7]:
import importlib
import core
importlib.reload(core)
from core import db, Question
from core import parse_key, get_key, key_to_query_filter

In [5]:
from core import get_categories
get_categories('difficulty:easy')

In [2]:
# DB Connect
db = SqliteDatabase('data/index.db')
db.connect()

In [15]:
# db.close()
# os.remove('data/index.db')

In [3]:
# Load data
data_path = Path('.').parent / 'data'
db_path = data_path / r"cb-digital-questions.json"
with db_path.open(encoding='utf-8') as fp:
    questions = json.load(fp)

with (data_path / r"lookup.json").open(encoding='utf-8') as fp:
    filter_data = json.load(fp)

    for k, v in [('mathLiveItems', 'math'), ('readingLiveItems', 'english')]:
        filter_data[v] = filter_data[k]
        del filter_data[k]

question_id_map = {}
for q_id, q in questions.items():
    question_id_map[q['questionId']] = q_id

In [31]:
# Create Tables
db.drop_tables([Question])
db.create_tables([Question])

In [32]:
# Add Question to DB
questions_model = []
for q_id, q_meta in questions.items():
    question = Question(
        q_id=q_id,
        questionId=q_meta['questionId'],
        skill_cd=q_meta['skill_cd'],
        score_band_range_cd=q_meta['score_band_range_cd'],
        uId=q_meta['uId'],
        program=q_meta['program'],
        external_id=q_meta['external_id'],
        primary_class_cd=q_meta['primary_class_cd'],
        difficulty=q_meta['difficulty'],
        module=q_meta['module'],
        active=q_meta['external_id'] in filter_data[q_meta['module']]
    )
    questions_model.append(question)

Question.bulk_create(questions_model)

In [6]:
Question.select().where(Question.active == False).count()

In [7]:
def questions_filter(where = None) -> List[Question]:
    print(Question.select().where(where).count())
    
    # query = Question.select().where(
    #     Question.module == module,
    #     Question.skill_cd == skill_cd,
    #     Question.primary_class_cd == primary_class_cd,
    #     Question.active.in_(active)
    # )
    # 
    # return list(query)

In [8]:
questions_filter(module='math')

In [70]:


print(parse_key(''))

parse_key(get_key(module='math', primary_class_cd='CAS', skill_cd='all', difficulty='all', active='all'))

In [15]:
# category -> subcategory
# primary_class_cd -> skill_cd

query = (
    Question.select(
        Question.primary_class_cd.alias("primary_class"),
        Question.skill_cd.alias("skill"),
        # fn.COUNT(Question.primary_class_cd).filter(Question.primary_class_cd == Question.primary_class_cd).alias("primary_class_count"),
        # SQL("(SELECT Count(*) FROM question WHERE primary_class_cd = primary_class_cd) as primary_class_count"),
        (Question.select(SQL('COUNT(*) AS primary_class_count')).where(Question.primary_class_cd == Question.primary_class_cd).order_by(Question.primary_class_cd).group_by(Question.primary_class_cd)).alias("primary_class_count"),
        
        # fn.COUNT(Question.primary_class_cd).alias("primary_class_count"),
        fn.COUNT(Question.skill_cd).alias("skill_count"),
    )
    # .where(Question.difficulty == 'E')
    .order_by(Question.module, Question.primary_class_cd, Question.skill_cd)
    .group_by(Question.module, Question.skill_cd)
)

# Execute the query and iterate through results
for row in query:
    print(
        f"primary_class: {row.primary_class}, skill: {row.skill}, primary_class_count: {row.primary_class_count}, skill_count: {row.skill_count}"
    )

In [19]:
question_class_counts = (
    Question.select(
        Question.module, 
        Question.primary_class_cd, 
        fn.COUNT().alias('count'),
        Question.questionId.alias('first_question_id_class')
    )
    # .where(Question.difficulty == 'E')
    .order_by(Question.module, Question.primary_class_cd)
    .group_by(Question.primary_class_cd)
)

for question in question_class_counts:
    print(f"Module: {question.module}, Class: {question.primary_class_cd}, Count: {question.count}, QuestionId: {question.first_question_id_class}")

In [71]:
def questions_id(filter_s: str = '') -> List[str]:
    query = Question.select(Question.questionId).where(
        *(key_to_query_filter(filter_s) or [None])
    )
    return [q.questionId for q in query]

def questions_count(filter_s: str = '') -> int:
    return Question.select(Question.questionId).where(
        *(key_to_query_filter(filter_s) or [None])
    ).count()

len(questions_id('module:math-class:all-skill:all-difficulty:all-active:bluebook'))

In [13]:
Question.select().where(Question.module=='english', Question.active==None).count()

In [14]:
def a(**kwargs):
    print(kwargs)
    
a(class=1)